In [ ]:
#TODO: This notebook will preprocess the data from the dataset containing all historic data of all risk factors to be in a format that Prophet can work with

In [ ]:
import pandas as pd
import requests
import logging
from logging import log, INFO, ERROR, WARN, DEBUG
import json
from matplotlib import pyplot as plt
import io

In [ ]:
class restcall():
    steps = 0
    training = False

    def forecast(steps, df):
        logging.basicConfig(level=INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.info("Number of chosen forecast steps: " + str(steps))

        # Parameters
        algorithm = 'getProphetForecast'
        url = 'http://localhost:8061/' + algorithm

        logging.info("Read data ...")
        logging.info("POST request startet ...")

        # Prepare POST request
        jsn = {
            "steps": steps,
            "data": df.to_json(orient='records')
        }
        response = json.dumps(jsn)
        logging.info("Ready for POST ...")

        # Send POST and retrieve response
        req = requests.post(url,response).content
        logging.info(req)
        output = pd.read_json(io.StringIO(req.decode('utf-8')))

        return output

In [ ]:
# Prepare the Dataset
df_historic = pd.read_csv("../DataAnalysis/Data/preprocessed_data.csv")
#df_historic = df_historic.drop('Unnamed: 0', axis=1)
df_historic['Date'] = pd.to_datetime(df_historic.Date)
df_historic['Date'] = df_historic['Date'].dt.strftime('%m/%d/%Y')
# add indicator for historic and forecasted values
df_historic['historic'] = 1
df_historic

In [ ]:
# Iterate over all columns and create tuples of date and feature
for column in df_historic:
    # Skip for Date and Index columns
    if column == 'Date' or column == 'historic':
        continue
    # Prepare the tuple that is sent to the API
    df_prophet = df_historic[['Date', column]]
    # Retrieve forecasts
    forecasts = restcall.forecast(365, df_prophet)
    forecasts['DATE'] = pd.to_datetime(forecasts.DATE)
    forecasts['DATE'] = forecasts['DATE'].dt.strftime('%m/%d/%Y')
    forecasts = forecasts.rename(columns={'DATE':'Date'})
    forecasts = forecasts.rename(columns={'Y': column})
    
    # If first iteration (column 'Price'), add forecast dates to our final dataset
    if column == 'Price':
        df_forecast = pd.DataFrame()
        df_forecast['Date'] = forecasts['Date']
    
    # Inner join the forecasts on date
    df_forecast = pd.merge(df_forecast, forecasts, on='Date')

df_forecast.head()

In [ ]:
# Indicate that these are forecasted values
df_forecast['historic'] = 0
df_forecast.head()

In [ ]:
df_final = pd.concat([df_historic, df_forecast], axis=0)
df_final = df_final.reset_index(drop=True)
df_final.tail(92)

In [ ]:
df_final.to_csv('./hist_forecast.csv')